# AugmentedGPT Playground

Please run `poetry install` under the project root directory. This will install all the necessary dependencies for you.

## Imports and set API key

In [1]:
from augmented_gpt import *
import getpass
API_KEY = getpass.getpass('Enter your OpenAI API key: ')

## Create a GPT instance that can use tools, and remembers the conversation in a long-term memory storage.

In [2]:
@function
def send_email(
    topic: str = param("The email topic"),
    to: str = param("The recipient email address"),
    content: str = param("The content of the email"),
):
    """Send an email to a recipient"""
    gpt.logger.debug(f"➡️ send_email to {to}")
    print(f"---")
    print(f"TOPIC: {topic}")
    print(f"TO: {to}")
    print(f"CONTENT:\n\n{content}")
    print(f"---")
    return "done."

gpt = AugmentedGPT(api_key=API_KEY, functions=[send_email], plugins=plugins.all_plugins(), debug=True)

gpt.get_plugin("Memory").clear_memory() # type: ignore

completion = gpt.chat_completion([
    Message(role=Role.USER, content="Steve was wondering what is the distance between moon and earth \
        divided by two. Figure this out and send him an email."),
])

async for response in completion:
    print(response)

[INFO:AugmentedGPT] Register-Function: send_email
[INFO:AugmentedGPT] Register-Function: Timestamp-get_current_timestamp
[INFO:AugmentedGPT] Register-Function: Memory-remember
[INFO:AugmentedGPT] Register-Function: Memory-search_from_memory
[INFO:AugmentedGPT] Register-Function: Calculator-evaluate


Message(role=<Role.ASSISTANT: 'assistant'>, content=None, name=None, function_call=None, tool_calls=[ToolCall(id='call_82GswTxaijpe1pbykxIqyHUq', function=FunctionCall(name='Calculator-evaluate', arguments={'expression': '384400 / 2'}), type='function')], tool_call_id=None)


[DEBUG:AugmentedGPT] ➡️ Calculator-evaluate: expression=384400 / 2


Message(role=<Role.TOOL: 'tool'>, content='192200.0', name=None, function_call=None, tool_calls=[], tool_call_id='call_82GswTxaijpe1pbykxIqyHUq')
Message(role=<Role.ASSISTANT: 'assistant'>, content=None, name=None, function_call=None, tool_calls=[ToolCall(id='call_j1aiMjmoz0Cyawimgq9TIs1M', function=FunctionCall(name='Timestamp-get_current_timestamp', arguments={}), type='function'), ToolCall(id='call_3e0Aq2RsilplKcr9UmEO2o99', function=FunctionCall(name='Memory-search_from_memory', arguments={'query': "Steve's email address"}), type='function')], tool_call_id=None)


[DEBUG:AugmentedGPT] ➡️ Timestamp-get_current_timestamp: 
[DEBUG:AugmentedGPT] ➡️ Memory-search_from_memory: query=Steve's email address


Message(role=<Role.TOOL: 'tool'>, content='2023-11-11T20:32:10.420417', name=None, function_call=None, tool_calls=[], tool_call_id='call_j1aiMjmoz0Cyawimgq9TIs1M')
Message(role=<Role.TOOL: 'tool'>, content='TIMESTAMP: 2023-11-11T20:32:05.312556\nCONTENT:\n{"role": "user", "content": "Steve was wondering what is the distance between moon and earth         divided by two. Figure this out and send him an email."}\n\n---\n\nTIMESTAMP: 2023-11-11T20:32:07.115465\nCONTENT:\n{"role": "assistant", "content": null}\n\n---\n\nTIMESTAMP: 2023-11-11T20:32:10.417463\nCONTENT:\n{"role": "assistant", "content": null}\n\n---\n\n', name=None, function_call=None, tool_calls=[], tool_call_id='call_3e0Aq2RsilplKcr9UmEO2o99')
Message(role=<Role.ASSISTANT: 'assistant'>, content="I am currently unable to find Steve's email address as there is no mention of it in our previous conversations or stored knowledge. To proceed with sending an email to Steve, I will need to know his email address. Can you please pro

## Create another GPT instance that inherits and utilizes the memory.

In [3]:
gpt = AugmentedGPT(functions=[send_email], plugins=plugins.all_plugins(), debug=True)

# Make it retrieve the answer from the memorized conversation history
completion = gpt.chat_completion([
    Message(role=Role.USER, content="Ok Steve asked for the Moon and Earth distance question again \
        after 10 days! Email him the answer if you can still remember, and tell him \
        he already asked for this before!"),
])

async for message in completion:
    print(message)

[INFO:AugmentedGPT] Register-Function: send_email
[INFO:AugmentedGPT] Register-Function: Timestamp-get_current_timestamp
[INFO:AugmentedGPT] Register-Function: Memory-remember
[INFO:AugmentedGPT] Register-Function: Memory-search_from_memory
[INFO:AugmentedGPT] Register-Function: Calculator-evaluate


Message(role=<Role.ASSISTANT: 'assistant'>, content=None, name=None, function_call=None, tool_calls=[ToolCall(id='call_R9XBPiwJZHL4ZodM35lf3C1D', function=FunctionCall(name='Memory-search_from_memory', arguments={'query': 'Moon and Earth distance question from Steve'}), type='function')], tool_call_id=None)


[DEBUG:AugmentedGPT] ➡️ Memory-search_from_memory: query=Moon and Earth distance question from Steve


Message(role=<Role.TOOL: 'tool'>, content='TIMESTAMP: 2023-11-11T20:32:13.946684\nCONTENT:\n{"role": "user", "content": "Ok Steve asked for the Moon and Earth distance question again         after 10 days! Email him the answer if you can still remember, and tell him         he already asked for this before!"}\n\n---\n\nTIMESTAMP: 2023-11-11T20:32:05.312556\nCONTENT:\n{"role": "user", "content": "Steve was wondering what is the distance between moon and earth         divided by two. Figure this out and send him an email."}\n\n---\n\nTIMESTAMP: 2023-11-11T20:32:11.394710\nCONTENT:\n{"role": "tool", "content": "TIMESTAMP: 2023-11-11T20:32:05.312556\\nCONTENT:\\n{\\"role\\": \\"user\\", \\"content\\": \\"Steve was wondering what is the distance between moon and earth         divided by two. Figure this out and send him an email.\\"}\\n\\n---\\n\\nTIMESTAMP: 2023-11-11T20:32:07.115465\\nCONTENT:\\n{\\"role\\": \\"assistant\\", \\"content\\": null}\\n\\n---\\n\\nTIMESTAMP: 2023-11-11T20:32:1

[DEBUG:AugmentedGPT] ➡️ Memory-search_from_memory: query=Steve's email


Message(role=<Role.TOOL: 'tool'>, content='TIMESTAMP: 2023-11-11T20:32:13.614430\nCONTENT:\n{"role": "assistant", "content": "I am currently unable to find Steve\'s email address as there is no mention of it in our previous conversations or stored knowledge. To proceed with sending an email to Steve, I will need to know his email address. Can you please provide it?"}\n\n---\n\nTIMESTAMP: 2023-11-11T20:32:05.312556\nCONTENT:\n{"role": "user", "content": "Steve was wondering what is the distance between moon and earth         divided by two. Figure this out and send him an email."}\n\n---\n\nTIMESTAMP: 2023-11-11T20:32:13.946684\nCONTENT:\n{"role": "user", "content": "Ok Steve asked for the Moon and Earth distance question again         after 10 days! Email him the answer if you can still remember, and tell him         he already asked for this before!"}\n\n---\n\n', name=None, function_call=None, tool_calls=[], tool_call_id='call_GSM0CcruDbMm95iyiwS1uJ9X')
Message(role=<Role.ASSISTANT: 